In [1]:
import pandas_gbq as pdq
import pandas as pd
import pydata_google_auth
import win32com.client as win32
from datetime import datetime as dt, timedelta
date = dt.today()- timedelta(days=dt.today().day)
date = date.strftime('%m.%Y')
namefile = 'Monthly Data Request ' + date + '.xlsx'
SCOPES = [
    'https://www.googleapis.com/auth/cloud-platform',
    'https://www.googleapis.com/auth/drive',
]

credentials = pydata_google_auth.get_user_credentials(
    SCOPES,
    # Set auth_local_webserver to True to have a slightly more convienient
    # authorization flow. Note, this doesn't work if you're running from a
    # notebook on a remote sever, such as over SSH or with Google Colab.
    auth_local_webserver=True,
)

project = 'nbcu-ds-sandbox-a-001'

In [2]:
Max_Date = pdq.read_gbq("""SELECT MAX(PARSE_DATE('%Y%m%d', pdl_event_date)) - 1
        FROM `nbcu-ds-prod-001.PeacockOnSkyDataMart.pos_stb_raw`
        WHERE Territory IN('GB', 'IE')""",
project_id=project,
credentials=credentials)

Min_Date = pdq.read_gbq("""SELECT MAX(PARSE_DATE('%Y%m%d', pdl_event_date)) - 30
        FROM `nbcu-ds-prod-001.PeacockOnSkyDataMart.pos_stb_raw`
        WHERE Territory IN('GB', 'IE')""",
project_id=project,
credentials=credentials)

date_range = Min_Date.iloc[0,0].strftime("%m/%d") + ' - ' + Max_Date.iloc[0,0].strftime("%m/%d")
date_range

Downloading: 100%|█████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.73rows/s]


'05/05 - 06/03'

In [3]:
MAAs_Territory_Provider_Setup = pdq.read_gbq(
"""
    DECLARE periodEnd DATE;
DECLARE periodStart DATE;
DECLARE month INT64;
DECLARE year INT64;

SET month = 12;
SET year = 2021;
SET periodStart = DATE(year, month, 1);
SET periodEnd = DATE(year, month, 1) + 29;

-- Find the last date in the STB table to use to find trailing 30 days
CREATE TEMP FUNCTION STB_Max_Date()
AS(
    (SELECT MAX(PARSE_DATE('%Y%m%d', pdl_event_date)) - 1
    FROM `nbcu-ds-prod-001.PeacockOnSkyDataMart.pos_stb_raw`
    WHERE Territory IN('GB', 'IE'))
);

DROP TABLE IF EXISTS `nbcu-ds-sandbox-a-001.wes_sandbox.MonthlyMAAsByTerritoryAndProvider`;
CREATE TABLE `nbcu-ds-sandbox-a-001.wes_sandbox.MonthlyMAAsByTerritoryAndProvider`(
  Start_Of_Period DATE,
  End_Of_Period DATE,
  Provider STRING,
  Territory STRING,
  MAAs INT64
);

LOOP
  IF periodEnd > STB_Max_Date() THEN
    BREAK;
  ELSE
    INSERT INTO `nbcu-ds-sandbox-a-001.wes_sandbox.MonthlyMAAsByTerritoryAndProvider`
    WITH SkyTV AS(
        SELECT
            "STB" AS Provider,
            Territory,
            PARSE_DATE('%Y%m%d', pdl_event_date) AS Watch_Date,
            customer_id AS ProfileId,
            '' AS ContentTitle
        FROM `nbcu-ds-prod-001.PeacockOnSkyDataMart.pos_stb_raw`
        WHERE customer_id NOT IN(SELECT customer_id FROM `nbcu-ds-prod-001.PeacockOnSkyDataMart.pos_stb_raw` WHERE Territory = 'IT' AND pdl_event_date < '20220214')
    ),

    OTT AS(
        SELECT 
            "OTT" AS Provider,
            ProviderTerritory AS Territory,
            DATE(streamingSessionStart) AS Watch_Date,
            ProfileId,
            ContentTitle
        FROM `nbcu-ds-prod-001.PeacockOnSkyDataMart.pos_ott_raw`
        WHERE ProgrammeViewingDuration >= 120
    ),

    Combined AS(
        SELECT * FROM OTT 
        UNION ALL 
        SELECT * FROM SkyTV 
    ),

    Intl AS(
        SELECT * EXCEPT(Territory), CASE WHEN Territory IN('GB', 'IE') THEN 'UK' WHEN Territory IN('DE', 'AT') THEN 'DE' WHEN Territory IN('IT') THEN 'IT' END Territory
        FROM Combined
        WHERE Combined.Watch_Date BETWEEN periodEnd - 29 AND periodEnd
    )
                
    
    SELECT
      periodStart AS Start_of_Period,
      periodEnd AS End_of_Period,
      Provider,
      Territory,
      COUNT(DISTINCT(ProfileId)) AS MAAs
    FROM Intl
    GROUP BY Provider, Territory
    UNION ALL
    (SELECT 
      periodEnd - 29 AS Start_of_Period,
      periodEnd AS End_of_Period,
      Provider,
      'Total' Territory,
      COUNT(DISTINCT(ProfileId)) AS MAAs
    FROM Intl
    GROUP BY Provider, Territory)
    UNION ALL
    (SELECT 
      periodEnd - 29 AS Start_of_Period,
      periodEnd AS End_of_Period,
      'Total' Provider,
      'Total' Territory,
      COUNT(DISTINCT(ProfileId)) AS MAA
    FROM Intl
    GROUP BY Provider, Territory)
    ORDER BY Provider, MAAs, Territory;

    IF month = 12 THEN
      SET month = 1;
      SET year = year + 1;
    ELSE
      SET month = month + 1;
    END IF;

    SET periodStart = DATE(year, month, 1);
    SET periodEnd = DATE(year, month, 1) + 29;

  END IF;
END LOOP;

""",
project_id=project,
credentials=credentials
)

In [4]:
MAAs_Territory_Provider = pdq.read_gbq(
    """
    SELECT *
    FROM `nbcu-ds-sandbox-a-001.wes_sandbox.MonthlyMAAsByTerritoryAndProvider`
    ORDER BY 
        Start_Of_Period, Provider, 
        CASE 
            WHEN Territory = 'IT' THEN 1 
            WHEN Territory = 'DE' THEN 2  
            WHEN Territory = 'UK' THEN 3  
            WHEN Territory = 'Total' THEN 4 
        END
    """,
project_id=project,
credentials=credentials)

Downloading: 100%|██████████████████████████████████████████████████████████████████| 48/48 [00:00<00:00, 132.47rows/s]


In [5]:
Daily_AAs_STB = pdq.read_gbq(
    """
    -- Find the last date in the STB table to use to find trailing 30 days
    CREATE TEMP FUNCTION STB_Max_Date()
    AS(
        (
            SELECT MAX(PARSE_DATE('%Y%m%d', pdl_event_date)) - EXTRACT(Day FROM MAX(PARSE_DATE('%Y%m%d', pdl_event_date)))
            FROM `nbcu-ds-prod-001.PeacockOnSkyDataMart.pos_stb_raw`
            WHERE Territory IN('GB', 'IE')
        )
    );

    SELECT 
        Watch_Date, 
        Territory, 
        COUNT(DISTINCT(customer_id)) MAAs
    FROM`nbcu-ds-sandbox-a-001.wes_sandbox.STB`
    WHERE Watch_Date <= STB_Max_Date()
    GROUP BY Watch_Date, Territory
    ORDER BY Watch_Date ASC, Territory ASC
    """,
project_id=project,
credentials=credentials)

Downloading: 100%|███████████████████████████████████████████████████████████████| 435/435 [00:00<00:00, 1197.92rows/s]


In [6]:
Daily_AAs_OTT = pdq.read_gbq(
    """
    -- Find the last date in the STB table to use to find trailing 30 days
    CREATE TEMP FUNCTION STB_Max_Date()
    AS(
        (
            SELECT MAX(PARSE_DATE('%Y%m%d', pdl_event_date)) - EXTRACT(Day FROM MAX(PARSE_DATE('%Y%m%d', pdl_event_date)))
            FROM `nbcu-ds-prod-001.PeacockOnSkyDataMart.pos_stb_raw`
            WHERE Territory IN('GB', 'IE')
        )
    );

    SELECT 
        Watch_Date, 
        Territory, 
        COUNT(DISTINCT(ProfileId)) MAAs
    FROM`nbcu-ds-sandbox-a-001.wes_sandbox.OTT_With_Titles`
    WHERE 
        ProgrammeViewingDuration >= 120
        AND Watch_Date <= STB_Max_Date()
    GROUP BY Watch_Date, Territory
    ORDER BY Watch_Date ASC, Territory
    """
)

Downloading: 100%|███████████████████████████████████████████████████████████████| 436/436 [00:00<00:00, 1066.41rows/s]


In [7]:
MAAs_By_TerritoryAndContentTitle_Setup = pdq.read_gbq(
"""
    DECLARE periodEnd DATE;
    DECLARE periodStart DATE;
    DECLARE month INT64;
    DECLARE year INT64;

    SET month = 12;
    SET year = 2021;
    SET periodStart = DATE(year, month, 1);
    SET periodEnd = DATE(year, month, 1) + 29;

    -- Find the last date in the STB table to use to find trailing 30 days
    CREATE TEMP FUNCTION STB_Max_Date()
    AS(
        (SELECT MAX(PARSE_DATE('%Y%m%d', pdl_event_date)) - 1
        FROM `nbcu-ds-prod-001.PeacockOnSkyDataMart.pos_stb_raw`
        WHERE Territory IN('GB', 'IE'))
    );

    DROP TABLE IF EXISTS `nbcu-ds-sandbox-a-001.wes_sandbox.MonthlyMAAsByTerritoryAndContentTitle`;
    CREATE TABLE `nbcu-ds-sandbox-a-001.wes_sandbox.MonthlyMAAsByTerritoryAndContentTitle`(
      Start_Of_Period DATE,
      End_Of_Period DATE,
      Territory STRING,
      ContentTitle STRING,
      MAAs INT64
    );

    LOOP
      IF periodEnd > STB_Max_Date() THEN
        BREAK;
      ELSE
        INSERT INTO `nbcu-ds-sandbox-a-001.wes_sandbox.MonthlyMAAsByTerritoryAndContentTitle`
        SELECT
          periodStart Start_Of_Period,
          periodEnd End_Of_Period,
          Territory,
          ContentTitle,
          COUNT(DISTINCT(ProfileId)) MAAs
        FROM `nbcu-ds-sandbox-a-001.wes_sandbox.OTT_With_Titles`
        WHERE 
          ProgrammeViewingDuration >= 120
          AND Watch_Date BETWEEN periodStart AND periodEnd
        GROUP BY Territory, ContentTitle
        ORDER BY Territory, ContentTitle;

        IF month = 12 THEN
          SET month = 1;
          SET year = year + 1;
        ELSE
          SET month = month + 1;
        END IF;

        SET periodStart = DATE(year, month, 1);
        SET periodEnd = DATE(year, month, 1) + 29;

      END IF;
    END LOOP;
""",
project_id=project,
credentials=credentials)

In [8]:
MAAs_By_TerritoryAndContentTitle = pdq.read_gbq(
"""
    SELECT *
    FROM `nbcu-ds-sandbox-a-001.wes_sandbox.MonthlyMAAsByTerritoryAndContentTitle`
    ORDER BY Start_Of_Period, Territory, ContentTitle
""",
project_id=project,
credentials=credentials)

Downloading: 100%|█████████████████████████████████████████████████████████████| 1756/1756 [00:00<00:00, 3608.94rows/s]


In [9]:
Daily_AAs_Starts_Usage = pdq.read_gbq(
    """
    -- Find the last date in the STB table to use to find trailing 30 days
    CREATE TEMP FUNCTION STB_Max_Date()
    AS(
        (SELECT MAX(PARSE_DATE('%Y%m%d', pdl_event_date)) - EXTRACT(Day FROM MAX(PARSE_DATE('%Y%m%d', pdl_event_date)))
        FROM `nbcu-ds-prod-001.PeacockOnSkyDataMart.pos_stb_raw`
        WHERE Territory IN('GB', 'IE'))
    );

    SELECT 
        Territory,
        Watch_Date,
        ContentTitle,
        COUNT(DISTINCT(ProfileID)) AAs,
        COUNT(ProfileID) ContentStarts,
        ROUND(SUM(ProgrammeViewingDuration)/3600, 1) Usage
    FROM `nbcu-ds-sandbox-a-001.wes_sandbox.OTT_With_Titles` 
    WHERE 
        ProgrammeViewingDuration >= 120 
        AND Watch_Date <=  STB_Max_Date()
    GROUP BY 1,2,3
    ORDER BY 1 DESC,2,3
    """,
project_id=project,
credentials=credentials)

Daily_AAs_Starts_Usage['Usage'] = Daily_AAs_Starts_Usage['Usage'].astype(float)

Downloading: 100%|██████████████████████████████████████████████████████████| 46607/46607 [00:04<00:00, 11182.32rows/s]


In [10]:
OTT_MAAs_Days_Watched_Setup = pdq.read_gbq(
"""
DECLARE periodEnd DATE;
DECLARE periodStart DATE;
DECLARE month INT64;
DECLARE year INT64;

SET month = 12;
SET year = 2021;
SET periodStart = DATE(year, month, 1);
SET periodEnd = DATE(year, month, 1) + 29;

-- Find the last date in the STB table to use to find trailing 30 days
CREATE TEMP FUNCTION STB_Max_Date()
AS(
    (SELECT MAX(PARSE_DATE('%Y%m%d', pdl_event_date)) - 1
    FROM `nbcu-ds-prod-001.PeacockOnSkyDataMart.pos_stb_raw`
    WHERE Territory IN('GB', 'IE'))
);

DROP TABLE IF EXISTS `nbcu-ds-sandbox-a-001.wes_sandbox.MAAsPerDaysUsed`;
CREATE TABLE `nbcu-ds-sandbox-a-001.wes_sandbox.MAAsPerDaysUsed`(
    Start_Of_Period DATE,
    End_Of_Period DATE,
    Days_Watched INT64,
    UK INT64,
    DE INT64,
    IT INT64
);
LOOP
    IF periodENd > STB_Max_Date() THEN
        BREAK;
    ELSE
        INSERT INTO `nbcu-ds-sandbox-a-001.wes_sandbox.MAAsPerDaysUsed`
        WITH Setup AS(
            SELECT
                periodStart AS Start_of_Period,
                periodEnd AS End_of_Period,
                Days_Watched,
                Territory,
                COUNT(DISTINCT(ProfileID)) MAAs
            FROM(
                SELECT 
                    *,
                    CASE WHEN ProviderTerritory IN('GB', 'IE') THEN 'UK' WHEN ProviderTerritory IN('DE', 'AT') THEN 'DE' WHEN ProviderTerritory IN('IT') THEN 'IT' END Territory,
                    COUNT(DISTINCT(Watch_Date)) OVER(PARTITION BY ProfileID) Days_Watched
                FROM(
                    SELECT 
                        *, 
                        DATE(streamingSessionStart) Watch_Date,
                    FROM `nbcu-ds-prod-001.PeacockOnSkyDataMart.pos_ott_raw`
                    WHERE 
                        ProgrammeViewingDuration >= 120
                        AND DATE(streamingSessionStart) BETWEEN periodStart AND PeriodEnd
                )
            )
            GROUP BY Days_Watched, Territory
            ORDER BY Days_Watched
        )

        SELECT 
            Start_of_Period, End_of_Period, Days_Watched, UK, DE, IT
        FROM(
            SELECT *
            FROM Setup
        ) AS p 
        PIVOT(
            SUM(MAAs) FOR Territory IN('UK', 'DE', 'IT')
        ) p2
        ORDER BY Days_Watched;

        IF month = 12 THEN
        SET month = 1;
        SET year = year + 1;
        ELSE
        SET month = month + 1;
        END IF;

        SET periodStart = DATE(year, month, 1);
        SET periodEnd = DATE(year, month, 1) + 29;

    END IF;
END LOOP;
""",
project_id=project,
credentials=credentials
)

In [11]:
OTT_MAAs_Days_Watched = pdq.read_gbq(
    """
    SELECT *
    FROM `nbcu-ds-sandbox-a-001.wes_sandbox.MAAsPerDaysUsed`
    ORDER BY Start_Of_Period, Days_Watched
    """,
project_id=project,
credentials=credentials)

Downloading: 100%|████████████████████████████████████████████████████████████████| 180/180 [00:00<00:00, 445.18rows/s]


In [12]:
OTT_MAAs_Titles_Watched_Setup = pdq.read_gbq(
"""
DECLARE periodEnd DATE;
DECLARE periodStart DATE;
DECLARE month INT64;
DECLARE year INT64;

SET month = 12;
SET year = 2021;
SET periodStart = DATE(year, month, 1);
SET periodEnd = DATE(year, month, 1) + 29;

-- Find the last date in the STB table to use to find trailing 30 days
CREATE TEMP FUNCTION STB_Max_Date()
AS(
    (SELECT MAX(PARSE_DATE('%Y%m%d', pdl_event_date)) - 1
    FROM `nbcu-ds-prod-001.PeacockOnSkyDataMart.pos_stb_raw`
    WHERE Territory IN('GB', 'IE'))
);

DROP TABLE IF EXISTS `nbcu-ds-sandbox-a-001.wes_sandbox.TitlesPerMAA`;
CREATE TABLE `nbcu-ds-sandbox-a-001.wes_sandbox.TitlesPerMAA`(
    Start_Of_Period DATE,
    End_Of_Period DATE,
    Titles_Watched INT64,
    UK INT64,
    DE INT64,
    IT INT64
);
LOOP
    IF periodENd > STB_Max_Date() THEN
        BREAK;
    ELSE
        INSERT INTO `nbcu-ds-sandbox-a-001.wes_sandbox.TitlesPerMAA`
        WITH Setup AS(
            SELECT 
                periodStart AS Start_of_Period,
                periodEnd AS End_of_Period,
                Titles_Watched,
                Territory,
                COUNT(DISTINCT(ProfileID)) MAAs
            FROM(
                SELECT 
                    *,
                    CASE WHEN ProviderTerritory IN('GB', 'IE') THEN 'UK' WHEN ProviderTerritory IN('DE', 'AT') THEN 'DE' WHEN ProviderTerritory IN('IT') THEN 'IT' END Territory,
                    COUNT(DISTINCT(ContentTitle)) OVER(PARTITION BY ProfileID) Titles_Watched
                FROM(
                    SELECT 
                        *, 
                        DATE(streamingSessionStart) Watch_Date,
                    FROM `nbcu-ds-prod-001.PeacockOnSkyDataMart.pos_ott_raw`
                    WHERE 
                        ProgrammeViewingDuration >= 120
                        AND DATE(streamingSessionStart) BETWEEN periodStart AND periodEnd
                )
            )
            GROUP BY Titles_Watched, Territory
            ORDER BY Titles_Watched
        )

        SELECT 
            Start_of_Period, End_of_Period, Titles_Watched, UK, DE, IT
        FROM(
            SELECT *
            FROM Setup
        ) AS p 
        PIVOT(
            SUM(MAAs) FOR Territory IN('UK', 'DE', 'IT')
        ) p2
        ORDER BY Titles_Watched;

        IF month = 12 THEN
        SET month = 1;
        SET year = year + 1;
        ELSE
        SET month = month + 1;
        END IF;

        SET periodStart = DATE(year, month, 1);
        SET periodEnd = DATE(year, month, 1) + 29;

    END IF;
END LOOP;
""",
project_id=project,
credentials=credentials
)

In [13]:
OTT_MAAs_Titles_Watched = pdq.read_gbq(
    """
    SELECT *
    FROM `nbcu-ds-sandbox-a-001.wes_sandbox.TitlesPerMAA`
    ORDER BY Start_Of_Period, Titles_Watched
    """,
project_id=project,
credentials=credentials)

Downloading: 100%|████████████████████████████████████████████████████████████████| 118/118 [00:00<00:00, 307.47rows/s]


In [14]:
OTT_Category_Reach_Setup = pdq.read_gbq(
"""
DECLARE periodEnd DATE;
DECLARE periodStart DATE;
DECLARE month INT64;
DECLARE year INT64;

SET month = 12;
SET year = 2021;
SET periodStart = DATE(year, month, 1);
SET periodEnd = DATE(year, month, 1) + 29;

-- Find the last date in the STB table to use to find trailing 30 days
CREATE TEMP FUNCTION STB_Max_Date()
AS(
    (SELECT MAX(PARSE_DATE('%Y%m%d', pdl_event_date)) - 1
    FROM `nbcu-ds-prod-001.PeacockOnSkyDataMart.pos_stb_raw`
    WHERE Territory IN('GB', 'IE'))
);

DROP TABLE IF EXISTS `nbcu-ds-sandbox-a-001.wes_sandbox.CategoryReachDetailed`;
CREATE TABLE `nbcu-ds-sandbox-a-001.wes_sandbox.CategoryReachDetailed`(
    Start_Of_Period DATE,
    End_Of_Period DATE,
    Territory STRING,
    Category STRING,
    MAAs INT64
);

LOOP
    IF periodENd > STB_Max_Date() THEN
        BREAK;
    ELSE
        INSERT INTO `nbcu-ds-sandbox-a-001.wes_sandbox.CategoryReachDetailed`
        SELECT 
            periodStart AS Start_of_Period,
            periodEnd AS End_of_Period,
            c.Territory,
            c.Category, 
            COUNT(DISTINCT(ProfileId)) MAAs
        FROM(
            SELECT 
                *,
                CASE WHEN ProviderTerritory IN('GB', 'IE') THEN 'UK' WHEN ProviderTerritory IN('DE', 'AT') THEN 'DE' WHEN ProviderTerritory IN('IT') THEN 'IT' END Territory
            FROM `nbcu-ds-prod-001.PeacockOnSkyDataMart.pos_ott_raw` 
        ) o 
        LEFT JOIN `nbcu-ds-sandbox-a-001.wes_sandbox.ContentTitleCategories` c
        ON o.ContentTitle = c.ContentTitle AND o.Territory = c.Territory
        WHERE 
            ProgrammeViewingDuration >= 120
            AND DATE(streamingSessionStart) BETWEEN periodStart AND periodEnd
        GROUP BY c.Territory, Category
        UNION ALL
        SELECT
            periodStart AS Start_of_Period,
            periodEnd AS End_of_Period,
            c.Territory,
            CASE WHEN c.Category LIKE 'Library%' THEN 'Library' ELSE c.Category END Category, 
            COUNT(DISTINCT(ProfileId)) MAAs
        FROM(
            SELECT 
                *,
                CASE WHEN ProviderTerritory IN('GB', 'IE') THEN 'UK' WHEN ProviderTerritory IN('DE', 'AT') THEN 'DE' WHEN ProviderTerritory IN('IT') THEN 'IT' END Territory
            FROM `nbcu-ds-prod-001.PeacockOnSkyDataMart.pos_ott_raw` 
        ) o  
        LEFT JOIN (SELECT * FROM `nbcu-ds-sandbox-a-001.wes_sandbox.ContentTitleCategories` WHERE Category LIKE 'Library%') c
        ON o.ContentTitle = c.ContentTitle AND o.Territory = c.Territory
        WHERE 
            ProgrammeViewingDuration >= 120
            AND DATE(streamingSessionStart) BETWEEN periodStart AND PeriodEnd
            AND c.Category != ''
        GROUP BY c.Territory, Category
        ORDER BY Territory, Category;

                IF month = 12 THEN
                SET month = 1;
                SET year = year + 1;
                ELSE
                SET month = month + 1;
                END IF;

                SET periodStart = DATE(year, month, 1);
                SET periodEnd = DATE(year, month, 1) + 29;

    END IF;
END LOOP;
""",
project_id=project,
credentials=credentials
)

In [15]:
OTT_Category_Reach = pdq.read_gbq(
    """
   SELECT *
FROM `nbcu-ds-sandbox-a-001.wes_sandbox.CategoryReachDetailed`
ORDER BY Start_Of_Period, Category
""",
project_id=project,
credentials=credentials)


Downloading: 100%|████████████████████████████████████████████████████████████████| 108/108 [00:00<00:00, 278.54rows/s]


In [16]:
Gross_Adds_Analysis = pdq.read_gbq(
    """
    SELECT *
    FROM `nbcu-ds-sandbox-a-001.wes_sandbox.MonthlyintlGrossAddsLapsesReactivations`
    order by Start_of_Period, Territory, Provider
    """,
project_id=project,
credentials=credentials)

Downloading: 100%|███████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 77.96rows/s]


In [17]:
with pd.ExcelWriter(namefile) as filename:
    MAAs_Territory_Provider.to_excel(filename, sheet_name = 'MAAs by Territory and Provider', index = False)
    Daily_AAs_STB.to_excel(filename, sheet_name = 'Daily AAs STB', index = False)
    Daily_AAs_OTT.to_excel(filename, sheet_name = 'Daily AAs OTT', index = False)
    MAAs_By_TerritoryAndContentTitle.to_excel(filename, sheet_name = 'MAAs by ContentTitle', index = False)
    Daily_AAs_Starts_Usage.to_excel(filename, sheet_name = 'Daily AAs, Starts, Usage', index = False)
    OTT_MAAs_Days_Watched.to_excel(filename, sheet_name = '# of Days Used PoS', index = False)
    OTT_MAAs_Titles_Watched.to_excel(filename, sheet_name = '# of Titles Watched on PoS', index = False)
    OTT_Category_Reach.to_excel(filename, sheet_name='Category Reach', index = False)
    Gross_Adds_Analysis.to_excel(filename, sheet_name='Gross Adds Analysis', index = False)

In [18]:
outlook = win32.Dispatch('outlook.application')
mail = outlook.CreateItem(0)
mail.To = 'Shob.Kakkar@nbcuni.com'
mail.CC = 'Alejandra.Zannier@nbcuni.com; darragh.mulligan@nbcuni.com'
mail.Subject = 'Monthly Data Request ' + date
mail.Body = 'print(MAAs_Territory_Provider)'
mail.HTMLBody = (
    '''<body>
        <p>
            Hi All,
            <br><br>Please find the monthly data request attached. 
            <br><br>Please let me know if you have any questions. 
            <br><br>-Wes D.
        </p>
    </body>'''
    )
attachment  = 'C:/Users/206707903/Documents/Python/Shob_Request/' + namefile
mail.Attachments.Add(attachment)
mail.Save()
# mail.Send()